In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [6]:
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import *

In [7]:
x=df1.drop('target',axis=1)
y=df1['target']
xtr,xts,ytr,yts=train_test_split(x,y,test_size=0.3,random_state=123)

enc=CatBoostEncoder()
xtr_enc=enc.fit_transform(xtr,ytr)
xts_enc=enc.transform(xts)


In [8]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    print('AUC_ROC= ',auc)
    print('Accuracy= ',acc)
    print('confusion matrix:\n',confusion_matrix(y,pred))
    

In [9]:
lgb=LGBMClassifier()
params={'num_leaves': st.randint(10,50),
    'max_depth':st.randint(3,30) ,
    'learning_rate':st.uniform(0,1) ,
    'n_estimators':st.randint(100,250)}
sf=StratifiedKFold()
rs=RandomizedSearchCV(lgb,param_distributions=params,cv=sf,scoring='roc_auc',random_state=1234)
m=rs.fit(xtr_enc,ytr)
lbp=m.best_params_
print('RS:',m.best_score_)
print('params: ',lbp)
lgb=LGBMClassifier()
mod=lgb.fit(xtr_enc,ytr)
print('\nTrain:')
aucroc(mod,xtr_enc,ytr)
print('\nTest:')
aucroc(mod,xts_enc,yts)
print('classification report\n',classification_report(mod.predict(xts_enc),yts))


RS: 0.7919007813033507
params:  {'learning_rate': 0.022143913603027254, 'max_depth': 9, 'n_estimators': 239, 'num_leaves': 10}

Train:
AUC_ROC=  0.9589485821379052
Accuracy=  0.9562734327317116
confusion matrix:
 [[  371   955]
 [   48 21564]]

Test:
AUC_ROC=  0.8370478452792078
Accuracy=  0.9443596785677958
confusion matrix:
 [[  94  477]
 [  70 9190]]


In [10]:
print('classification report\n',classification_report(mod.predict(xts_enc),yts))

classification report
               precision    recall  f1-score   support

           0       0.16      0.57      0.26       164
           1       0.99      0.95      0.97      9667

    accuracy                           0.94      9831
   macro avg       0.58      0.76      0.61      9831
weighted avg       0.98      0.94      0.96      9831



In [13]:
rf=RandomForestClassifier()
params1={'min_samples_split': st.randint(2,100),
    'max_depth':st.randint(3,30) ,
    'min_samples_leaf':st.randint(1,50) ,
    'n_estimators':st.randint(100,250)
       }
sf=StratifiedKFold()
rs=RandomizedSearchCV(rf,param_distributions=params1,cv=sf,scoring='roc_auc',random_state=1234)
m=rs.fit(xtr_enc,ytr)
rbp=m.best_params_
print('RS:',m.best_score_)
print('params: ',rbp)
rf=RandomForestClassifier(**rbp)
mod1=rf.fit(xtr_enc,ytr)
print('\nTrain:')
aucroc(mod1,xtr_enc,ytr)
print('\nTest:')
aucroc(mod1,xts_enc,yts)
print('classification report\n',classification_report(mod1.predict(xts_enc),yts))


RS: 0.7903153450697762
params:  {'max_depth': 12, 'min_samples_leaf': 27, 'min_samples_split': 32, 'n_estimators': 224}

Train:
AUC_ROC=  0.8990713848431783
Accuracy=  0.9462464033481559
confusion matrix:
 [[  126  1200]
 [   33 21579]]

Test:
AUC_ROC=  0.8480242876541856
Accuracy=  0.9441562404638388
confusion matrix:
 [[  53  518]
 [  31 9229]]
classification report
               precision    recall  f1-score   support

           0       0.09      0.63      0.16        84
           1       1.00      0.95      0.97      9747

    accuracy                           0.94      9831
   macro avg       0.54      0.79      0.57      9831
weighted avg       0.99      0.94      0.96      9831



In [14]:
lr=RandomForestClassifier()
mod1=lr.fit(xtr_enc,ytr)
print('\nTrain:')
aucroc(mod1,xtr_enc,ytr)
print('\nTest:')
aucroc(mod1,xts_enc,yts)
print('classification report\n',classification_report(mod1.predict(xts_enc),yts))



Train:
AUC_ROC=  0.9999999999999999
Accuracy=  1.0
confusion matrix:
 [[ 1326     0]
 [    0 21612]]

Test:
AUC_ROC=  0.8277504699799147
Accuracy=  0.9454785881395585
confusion matrix:
 [[  93  478]
 [  58 9202]]
classification report
               precision    recall  f1-score   support

           0       0.16      0.62      0.26       151
           1       0.99      0.95      0.97      9680

    accuracy                           0.95      9831
   macro avg       0.58      0.78      0.61      9831
weighted avg       0.98      0.95      0.96      9831

